# Walmart 구매 데이터 분석 + 관계성(연관) 분석

이 노트북은 기존 Walmart 분석 흐름(전처리/가설검정/시각화)에 **관계성 분석(연관성 분석)** 파트를 추가한 버전입니다.

- **수치-수치**: 상관계수(스피어만/피어슨)
- **범주-범주**: 카이제곱 + **Cramér's V**
- **범주-수치(타깃 Purchase)**: **Correlation Ratio(η)**, Kruskal-Wallis
- **변수 중요도 관점**: Mutual Information, 간단 모델 기반 중요도

> 데이터 파일 경로만 맞추면 그대로 실행됩니다.


In [ ]:
# 0. 환경 설정
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import mutual_info_regression

sns.set_theme(style="whitegrid")
pd.set_option("display.max_columns", 200)


In [ ]:
# 1. 데이터 불러오기
# ✅ 아래 경로를 본인 환경에 맞게 수정하세요.
DATA_PATH = "Walmart.csv"   # 예: r"C:\Users\...\walmart.csv"

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"데이터 파일을 찾을 수 없습니다: {DATA_PATH}\n경로(DATA_PATH)를 수정하세요.")

df = pd.read_csv(DATA_PATH)
df.head()


In [ ]:
# 2. 기본 점검
df.info(), df.isna().sum()


## 3. 전처리 (카테고리 타입 정리)

보고서 흐름과 동일하게, 마스킹된 범주형 변수는 category로, 순서가 있는 변수는 순서형으로 정리합니다.


In [ ]:
# (필요 시) 컬럼명 공백/특수문자 정리
df.columns = [c.strip() for c in df.columns]

# 범주형 후보
cat_cols = []
for c in ["Gender", "Occupation", "City_Category", "Marital_Status", "Product_Category"]:
    if c in df.columns:
        cat_cols.append(c)

# Age / Stay는 데이터마다 형태가 다를 수 있어서 유연하게 처리
if "Age" in df.columns:
    cat_cols.append("Age")
if "Stay_In_Current_City_Years" in df.columns:
    cat_cols.append("Stay_In_Current_City_Years")

for c in cat_cols:
    df[c] = df[c].astype("category")

# 타깃 확인
assert "Purchase" in df.columns, "Purchase 컬럼이 필요합니다."
df["Purchase"] = pd.to_numeric(df["Purchase"], errors="coerce")

df.head()


## 4. 관계성 분석 A: 수치-수치 (상관)

수치형 컬럼들만 뽑아서 상관행렬을 봅니다. (Purchase 포함)  
- 분포가 비정규일 수 있어 **Spearman**도 함께 확인합니다.


In [ ]:
# 수치형 컬럼 추출
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols


In [ ]:
# 상관행렬 (Pearson)
corr_p = df[num_cols].corr(method="pearson")

plt.figure(figsize=(10, 6))
plt.title("Correlation (Pearson)")
sns.heatmap(corr_p, annot=False)
plt.show()

# 상관행렬 (Spearman)
corr_s = df[num_cols].corr(method="spearman")

plt.figure(figsize=(10, 6))
plt.title("Correlation (Spearman)")
sns.heatmap(corr_s, annot=False)
plt.show()


## 5. 관계성 분석 B: 범주-범주 (카이제곱 + Cramér's V)

범주형 변수끼리 **연관이 있는지**를 카이제곱으로 보고, 강도는 **Cramér's V (0~1)**로 봅니다.


In [ ]:
def cramers_v(confusion_matrix: pd.DataFrame) -> float:
    # Cramér's V 계산
    chi2 = stats.chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.to_numpy().sum()
    r, k = confusion_matrix.shape
    if n == 0:
        return np.nan
    return np.sqrt((chi2 / n) / (min(r - 1, k - 1) + 1e-12))

cat_cols_exist = df.select_dtypes(include=["category"]).columns.tolist()
cat_cols_exist


In [ ]:
# 범주-범주 조합별 Cramér's V 표
pairs = []
for i in range(len(cat_cols_exist)):
    for j in range(i+1, len(cat_cols_exist)):
        a, b = cat_cols_exist[i], cat_cols_exist[j]
        ct = pd.crosstab(df[a], df[b])
        v = cramers_v(ct)
        p = stats.chi2_contingency(ct)[1]
        pairs.append((a, b, v, p))

rel_cat_cat = pd.DataFrame(pairs, columns=["var1", "var2", "cramers_v", "p_value"]).sort_values("cramers_v", ascending=False)
rel_cat_cat.head(15)


## 6. 관계성 분석 C: 범주-수치 (Purchase 기준)

Purchase(수치)가 범주에 따라 달라지는지 확인합니다.
- **Correlation Ratio(η)**: 0~1 (클수록 해당 범주가 Purchase를 더 잘 '설명'함)
- 보조로 **Kruskal-Wallis**(비모수)로 유의성 확인


In [ ]:
def correlation_ratio(categories, measurements) -> float:
    # categories: array-like categorical
    # measurements: numeric array-like
    categories = pd.Series(categories).astype("category")
    measurements = pd.Series(measurements).astype(float)
    # drop NA pairs
    mask = categories.notna() & measurements.notna()
    categories = categories[mask]
    measurements = measurements[mask]

    if len(measurements) == 0:
        return np.nan

    y_mean = measurements.mean()
    ss_between = 0.0
    for lvl in categories.cat.categories:
        y_i = measurements[categories == lvl]
        if len(y_i) == 0:
            continue
        ss_between += len(y_i) * (y_i.mean() - y_mean) ** 2
    ss_total = ((measurements - y_mean) ** 2).sum()
    if ss_total == 0:
        return 0.0
    return np.sqrt(ss_between / ss_total)

cat_vs_purchase = []
for c in cat_cols_exist:
    eta = correlation_ratio(df[c], df["Purchase"])
    # Kruskal-Wallis
    groups = [df.loc[df[c] == lvl, "Purchase"].dropna().values for lvl in df[c].cat.categories]
    groups = [g for g in groups if len(g) > 0]
    if len(groups) >= 2:
        kw = stats.kruskal(*groups)
        p = kw.pvalue
    else:
        p = np.nan
    cat_vs_purchase.append((c, eta, p))

rel_cat_purchase = pd.DataFrame(cat_vs_purchase, columns=["category", "eta_correlation_ratio", "kruskal_p_value"])\
    .sort_values("eta_correlation_ratio", ascending=False)
rel_cat_purchase


## 7. 관계성 분석 D: Mutual Information (Purchase 기준)

비선형 관계까지 어느 정도 잡아주는 지표입니다. (값이 클수록 정보량이 많음)  
범주형은 순서 부여로 인코딩해서 간단히 계산합니다.


In [ ]:
# 모델용 피처 구성
feature_cols = [c for c in df.columns if c != "Purchase"]

X = df[feature_cols].copy()
y = df["Purchase"].copy()

# 범주형 -> Ordinal Encoding
cat_features = X.select_dtypes(include=["category", "object"]).columns.tolist()
num_features = [c for c in X.columns if c not in cat_features]

enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X_enc = X.copy()
if len(cat_features) > 0:
    X_enc[cat_features] = enc.fit_transform(X_enc[cat_features].astype(str))

# 결측 처리(간단히 중앙값/ -1)
for c in num_features:
    X_enc[c] = pd.to_numeric(X_enc[c], errors="coerce")
    X_enc[c] = X_enc[c].fillna(X_enc[c].median())

if len(cat_features) > 0:
    X_enc[cat_features] = X_enc[cat_features].fillna(-1)

# Mutual Information
mi = mutual_info_regression(X_enc, y, random_state=42)
mi_df = pd.DataFrame({"feature": X_enc.columns, "mutual_info": mi}).sort_values("mutual_info", ascending=False)
mi_df.head(15)


## 8. 관계성 분석 E: 간단 모델 기반 중요도 (Permutation Importance)

"Purchase에 영향을 주는 변수"를 모델 관점으로도 한번 봅니다.  
목표는 예측 성능이 아니라 **어떤 변수가 설명에 기여하는지 감 잡기**입니다.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

pred = rf.predict(X_test)
print("MAE:", mean_absolute_error(y_test, pred))
print("R2 :", r2_score(y_test, pred))


In [ ]:
perm = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
imp = pd.DataFrame({"feature": X_enc.columns, "importance": perm.importances_mean})\
    .sort_values("importance", ascending=False)

imp.head(15)


In [ ]:
# 중요도 상위 15개 시각화
topn = 15
imp_top = imp.head(topn).iloc[::-1]

plt.figure(figsize=(10, 6))
plt.title(f"Permutation Importance Top {topn}")
plt.barh(imp_top["feature"], imp_top["importance"])
plt.xlabel("Importance (decrease in score)")
plt.show()


## 9. 요약 출력 (한 장 요약용)

- 범주-수치: η 상위
- 범주-범주: Cramér's V 상위
- MI / Permutation Importance 상위

발표용/보고서용으로 "관계가 강한 쌍"만 빠르게 뽑을 때 씁니다.


In [ ]:
print("### 범주 vs Purchase (eta) 상위 5")
display(rel_cat_purchase.head(5))

print("\n### 범주 vs 범주 (Cramér's V) 상위 5")
display(rel_cat_cat.head(5))

print("\n### Mutual Information 상위 5")
display(mi_df.head(5))

print("\n### Permutation Importance 상위 5")
display(imp.head(5))
